In [ ]:
!ls ./drive/MyDrive/CS163

Egg_Sex			       Export_dataset_October24			Retina_Fundus
Export_dataset_November24.zip  ODIR-5K_Training_cropped_normal_Dataset


In [ ]:
!unzip ./drive/MyDrive/CS163/Export_dataset_November24.zip -d ./unzipped

In [ ]:
# Processing the csv file

import os
import csv
import random
from random import shuffle
import math
from pathlib import Path
import pickle
from PIL import Image
import torchvision.transforms as transforms
import torch
import cv2
import numpy as np

random.seed(1839)

dir_to_data = "./unzipped/HH19/"


csv_name = "GT_labels_eggs_HH19.csv"
csv_path = os.path.join('./', csv_name)
data_points = list()
with open(csv_path, "r") as f:
    reader = csv.reader(f)
    next(reader)  # to skip the header line
    for row_number, row in enumerate(reader):
        data_point_egg = dict()

        data_point_egg['label'] = int(row[1])

        if row[6] == '.':
            img_name = f'img_{row[0]}.png'
        else:
            img_name = f'lowQ_img_{row[0]}.png'

        data_point_egg['path'] = os.path.join(dir_to_data, row[6], img_name)

        additional_info = dict()
        additional_info['id'] = int(row[2])
        additional_info['grade'] = int(row[3])
        additional_info['real'] = int(row[4])
        additional_info['stage'] = int(row[5])
        data_point_egg['additional_info'] = additional_info

        data_points.append(data_point_egg)

data_points = [data_point for data_point in data_points if os.path.exists(data_point['path'])]

print(f'Total number of data points in the constructed data structure: {len(data_points)}')  # Check if number of existing images are correct

Total number of data points in the constructed data structure: 1553


In [ ]:
count_hh19 = !ls -l ./unzipped/HH19/*.png 2>/dev/null | grep -c '^-'
count_hh19_lower_quality = !ls -l ./unzipped/HH19/Lower_quality/*.png 2>/dev/null | grep -c '^-'

print(f'Total number of existing HH19 images in directory: {int(count_hh19[0]) + int(count_hh19_lower_quality[0])}')

Total number of existing HH19 images in directory: 1553


In [ ]:
random.seed(1839)

data_points_male = [data_point for data_point in data_points if data_point['label'] == 0]
data_points_female = [data_point for data_point in data_points if data_point['label'] == 1]

print(len(data_points_male) / len(data_points))

def get_split_for_data_points(input_data_points):
    id_count_dictionary = dict()
    for data_point in input_data_points:
        id = data_point['additional_info']['id']
        id_count_dictionary[id] = id_count_dictionary.get(id, 0) + 1


    print(sum(id_count_dictionary.values()))

    id_count_dictionary = list(id_count_dictionary.items())
    print(id_count_dictionary[0:10])
    id_count_dictionary.sort()
    shuffle(id_count_dictionary)
    print(id_count_dictionary[0:10])

    dataset_size = sum([data[1] for data in id_count_dictionary])
    validation_size, test_size = math.ceil(0.125 * dataset_size), math.ceil(0.125 * dataset_size)
    train_size = dataset_size - validation_size - test_size

    img_counter = 0
    index = 0
    while img_counter < test_size:
        img_counter += id_count_dictionary[index][1]
        index += 1
    test_slice = slice(0, index)

    img_counter = 0
    index_old = index
    while img_counter < validation_size:
        img_counter += id_count_dictionary[index][1]
        index += 1
    validation_slice = slice(index_old, index)

    img_counter = 0
    index_old = index
    while index < len(id_count_dictionary):
        img_counter += id_count_dictionary[index][1]
        index += 1
    train_slice = slice(index_old, index)

    train_ids = [data[0] for data in id_count_dictionary[train_slice]]
    validation_ids = [data[0] for data in id_count_dictionary[validation_slice]]
    test_ids = [data[0] for data in id_count_dictionary[test_slice]]

    data_points_train = [data_point for data_point in data_points if data_point['additional_info']['id'] in train_ids]
    data_points_validation = [data_point for data_point in data_points if data_point['additional_info']['id'] in validation_ids]
    data_points_test = [data_point for data_point in data_points if data_point['additional_info']['id'] in test_ids]

    return data_points_train, data_points_validation, data_points_test, train_ids, validation_ids, test_ids

data_points_train_male, data_points_validation_male, data_points_test_male, train_ids_male, validation_ids_male, test_ids_male = get_split_for_data_points(data_points_male)

dataset_size = len(data_points_train_male) + len(data_points_validation_male) + len(data_points_test_male)
print(f'train_size: {len(data_points_train_male) / dataset_size}, test_size: {len(data_points_test_male) / dataset_size}, validation_size: {len(data_points_validation_male) / dataset_size}')

data_points_train_female, data_points_validation_female, data_points_test_female, train_ids_female, validation_ids_female, test_ids_female = get_split_for_data_points(data_points_female)

dataset_size = len(data_points_train_female) + len(data_points_validation_female) + len(data_points_test_female)
print(f'train_size: {len(data_points_train_female) / dataset_size}, test_size: {len(data_points_test_female) / dataset_size}, validation_size: {len(data_points_validation_female) / dataset_size}')

data_points_train = data_points_train_male + data_points_train_female
data_points_validation = data_points_validation_male + data_points_validation_female
data_points_test = data_points_test_male + data_points_test_female

dataset_size = len(data_points_train) + len(data_points_validation) + len(data_points_test)
print(f'train_size: {len(data_points_train) / dataset_size}, test_size: {len(data_points_test) / dataset_size}, validation_size: {len(data_points_validation) / dataset_size}')



# Check if same data exists in different parts of the split
for data_point in data_points:
    if (data_point in data_points_train) and (data_point in data_points_validation):
        print("ERROR!!!")
    if (data_point in data_points_train) and (data_point in data_points_test):
        print("ERROR!!!")
    if (data_point in data_points_validation) and (data_point in data_points_test):
        print("ERROR!!!")
# Check if there are duplicates in a set
print(any(data_points_train.count(x) > 1 for x in data_points_train))
print(any(data_points_validation.count(x) > 1 for x in data_points_validation))
print(any(data_points_test.count(x) > 1 for x in data_points_test))

0.5112685125563425
794
[(10, 1), (16, 1), (20, 1), (22, 1), (25, 1), (27, 1), (28, 1), (45, 1), (46, 1), (48, 1)]
[(610, 1), (147, 1), (544, 1), (1284, 1), (709, 2), (1084, 1), (1114, 1), (998, 1), (1065, 1), (1264, 2)]
train_size: 0.7468513853904282, test_size: 0.12720403022670027, validation_size: 0.12594458438287154
759
[(1, 1), (2, 1), (8, 1), (26, 1), (30, 1), (32, 1), (34, 1), (36, 1), (39, 1), (40, 1)]
[(174, 1), (412, 1), (236, 1), (1280, 1), (192, 1), (752, 3), (72, 1), (1226, 3), (172, 1), (723, 2)]
train_size: 0.7483530961791831, test_size: 0.12648221343873517, validation_size: 0.1251646903820817
train_size: 0.7475853187379266, test_size: 0.12685125563425628, validation_size: 0.12556342562781714
False
False
False


In [ ]:
random.seed(1839)

# Processing the images

img_size = 512
pickle_chunk_size = 5000

def load_and_process_image(file_path, img_size):
    try:
        # image = Image.open(file_path)
        # image.save('./image_sample.png')

        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        # cv2.imwrite('./image_sample.png', image)

        image = (image / 256).astype(np.uint8)  # 16-bit to 8-bit
        image = Image.fromarray(image)
        # image.save('./image_sample.png')

        shorter_edge = min(image.size)

        # transforms.ToTensor() is removed for the pickle size to be smaller
        preprocess = transforms.Compose([
            transforms.CenterCrop(shorter_edge),
            transforms.Resize((img_size, img_size))
        ])

        image = preprocess(image)
        return image

    except Exception as e:
        print(f'Error loading the image: {e}')
        return None


def chunks(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i+chunk_size]


mode = 'validation'
data_points = data_points_validation

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH19/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)

mode = 'testing'
data_points = data_points_test

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH19/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)

mode = 'training'
data_points = data_points_train

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH19/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)


Processing chunk entry no: 10
Processing chunk entry no: 20
Processing chunk entry no: 30
Processing chunk entry no: 40
Processing chunk entry no: 50
Processing chunk entry no: 60
Processing chunk entry no: 70
Processing chunk entry no: 80
Processing chunk entry no: 90
Processing chunk entry no: 100
Processing chunk entry no: 110
Processing chunk entry no: 120
Processing chunk entry no: 130
Processing chunk entry no: 140
Processing chunk entry no: 150
Processing chunk entry no: 160
Processing chunk entry no: 170
Processing chunk entry no: 180
Processing chunk entry no: 190
Processing chunk entry no: 10
Processing chunk entry no: 20
Processing chunk entry no: 30
Processing chunk entry no: 40
Processing chunk entry no: 50
Processing chunk entry no: 60
Processing chunk entry no: 70
Processing chunk entry no: 80
Processing chunk entry no: 90
Processing chunk entry no: 100
Processing chunk entry no: 110
Processing chunk entry no: 120
Processing chunk entry no: 130
Processing chunk entry no:

In [ ]:
print(train_ids_male)
print(train_ids_male_hh19)

[1221, 648, 386, 186, 976, 775, 903, 828, 486, 744, 603, 707, 854, 234, 38, 1003, 534, 724, 877, 1077, 930, 310, 635, 1224, 1013, 1292, 48, 28, 469, 684, 420, 1076, 218, 1234, 1010, 1169, 609, 330, 720, 353, 979, 1122, 1263, 1142, 22, 602, 113, 769, 166, 375, 292, 873, 785, 1196, 855, 868, 1233, 594, 421, 1303, 1216, 133, 860, 747, 1073, 729, 481, 1074, 751, 1051, 160, 97, 411, 794, 1183, 546, 359, 400, 409, 558, 730, 376, 929, 468, 963, 990, 574, 654, 318, 987, 913, 633, 874, 631, 588, 276, 235, 1230, 801, 846, 500, 1105, 1136, 492, 1002, 157, 245, 199, 535, 832, 307, 577, 935, 1036, 659, 4, 560, 1291, 61, 1127, 46, 939, 1212, 869, 262, 1050, 308, 975, 1048, 119, 1123, 693, 790, 932, 531, 1018, 933, 674, 740, 137, 20, 444, 134, 822, 168, 1000, 925, 149, 148, 798, 45, 95, 213, 580, 583, 1338, 1004, 102, 196, 655, 493, 71, 506, 756, 797, 1124, 1057, 619, 657, 111, 1159, 909, 1078, 576, 244, 830, 1186, 726, 263, 920, 922, 678, 459, 817, 89, 581, 373, 1009, 1069, 461, 16, 25, 84, 209, 460

In [ ]:


train_ids_male_hh19 = train_ids_male
print(len(train_ids_male_hh19))
train_ids_female_hh19 = train_ids_female
print(len(train_ids_female_hh19))

validation_ids_male_hh19 = validation_ids_male
print(len(validation_ids_male_hh19))
validation_ids_female_hh19 = validation_ids_female
print(len(validation_ids_female_hh19))

test_ids_male_hh19 = test_ids_male
print(len(test_ids_male_hh19))
test_ids_female_hh19 = test_ids_female
print(len(test_ids_female_hh19))



448
403
73
74
77
66


In [ ]:
dir_to_data = "./unzipped/HH25/"

csv_name = "GT_labels_eggs_HH25.csv"
csv_path = os.path.join('./', csv_name)
data_points = list()
with open(csv_path, "r") as f:
    reader = csv.reader(f)
    next(reader)  # to skip the header line
    for row_number, row in enumerate(reader):
        data_point_egg = dict()

        data_point_egg['label'] = int(row[1])

        if row[6] == '.':
            img_name = f'img_{row[0]}.png'
        else:
            img_name = f'lowQ_img_{row[0]}.png'

        data_point_egg['path'] = os.path.join(dir_to_data, row[6], img_name)

        additional_info = dict()
        additional_info['id'] = int(row[2])
        additional_info['grade'] = int(row[3])
        additional_info['real'] = int(row[4])
        additional_info['stage'] = int(row[5])
        data_point_egg['additional_info'] = additional_info

        data_points.append(data_point_egg)

data_points = [data_point for data_point in data_points if os.path.exists(data_point['path'])]

print(f'Total number of data points in the constructed data structure: {len(data_points)}')  # Check if number of existing images are correct

Total number of data points in the constructed data structure: 1220


In [ ]:
count_hh25 = !ls -l ./unzipped/HH25/*.png 2>/dev/null | grep -c '^-'
count_hh25_lower_quality = !ls -l ./unzipped/HH25/Lower_quality/*.png 2>/dev/null | grep -c '^-'

print(f'Total number of existing HH25 images in directory: {int(count_hh25[0]) + int(count_hh25_lower_quality[0])}')

Total number of existing HH25 images in directory: 1220


In [ ]:
random.seed(1839)

data_points_male = [data_point for data_point in data_points if data_point['label'] == 0]
data_points_female = [data_point for data_point in data_points if data_point['label'] == 1]

print(len(data_points_male) / len(data_points))

def get_split_for_data_points(input_data_points, train_hh19, validation_hh19, test_hh19):
    id_count_dictionary = dict()
    for data_point in input_data_points:
        id = data_point['additional_info']['id']
        id_count_dictionary[id] = id_count_dictionary.get(id, 0) + 1


    print(sum(id_count_dictionary.values()))

    id_count_dictionary = list(id_count_dictionary.items())
    id_count_dictionary.sort()
    shuffle(id_count_dictionary)

    in_train_hh19 = [item for item in id_count_dictionary if item[0] in train_hh19]
    id_count_dictionary = [item for item in id_count_dictionary if item not in in_train_hh19]

    in_validation_hh19 = [item for item in id_count_dictionary if item[0] in validation_hh19]
    id_count_dictionary = [item for item in id_count_dictionary if item not in in_validation_hh19]

    in_test_hh19 = [item for item in id_count_dictionary if item[0] in test_hh19]
    id_count_dictionary = [item for item in id_count_dictionary if item not in in_test_hh19]

    dataset_size = sum([data[1] for data in id_count_dictionary])
    validation_size, test_size = math.ceil(0.125 * dataset_size), math.ceil(0.125 * dataset_size)
    train_size = dataset_size - validation_size - test_size

    img_counter = 0
    index = 0
    while img_counter < test_size:
        img_counter += id_count_dictionary[index][1]
        index += 1
    test_slice = slice(0, index)

    img_counter = 0
    index_old = index
    while img_counter < validation_size:
        img_counter += id_count_dictionary[index][1]
        index += 1
    validation_slice = slice(index_old, index)

    img_counter = 0
    index_old = index
    while index < len(id_count_dictionary):
        img_counter += id_count_dictionary[index][1]
        index += 1
    train_slice = slice(index_old, index)

    train_ids = [data[0] for data in id_count_dictionary[train_slice]]
    validation_ids = [data[0] for data in id_count_dictionary[validation_slice]]
    test_ids = [data[0] for data in id_count_dictionary[test_slice]]

    data_points_train = [data_point for data_point in data_points if data_point['additional_info']['id'] in (train_ids + train_hh19)]
    data_points_validation = [data_point for data_point in data_points if data_point['additional_info']['id'] in (validation_ids + validation_hh19)]
    data_points_test = [data_point for data_point in data_points if data_point['additional_info']['id'] in (test_ids + test_hh19)]

    return data_points_train, data_points_validation, data_points_test, train_ids, validation_ids, test_ids

data_points_train_male, data_points_validation_male, data_points_test_male, train_ids_male, validation_ids_male, test_ids_male = get_split_for_data_points(data_points_male, train_ids_male_hh19, validation_ids_male_hh19, test_ids_male_hh19)

dataset_size = len(data_points_train_male) + len(data_points_validation_male) + len(data_points_test_male)
print(f'train_size: {len(data_points_train_male) / dataset_size}, test_size: {len(data_points_test_male) / dataset_size}, validation_size: {len(data_points_validation_male) / dataset_size}')

data_points_train_female, data_points_validation_female, data_points_test_female, train_ids_female, validation_ids_female, test_ids_female = get_split_for_data_points(data_points_female, train_ids_female_hh19, validation_ids_female_hh19, test_ids_female_hh19)

dataset_size = len(data_points_train_female) + len(data_points_validation_female) + len(data_points_test_female)
print(f'train_size: {len(data_points_train_female) / dataset_size}, test_size: {len(data_points_test_female) / dataset_size}, validation_size: {len(data_points_validation_female) / dataset_size}')

data_points_train = data_points_train_male + data_points_train_female
data_points_validation = data_points_validation_male + data_points_validation_female
data_points_test = data_points_test_male + data_points_test_female

dataset_size = len(data_points_train) + len(data_points_validation) + len(data_points_test)
print(f'train_size: {len(data_points_train) / dataset_size}, test_size: {len(data_points_test) / dataset_size}, validation_size: {len(data_points_validation) / dataset_size}')

print(len(data_points_train), len(data_points_validation), len(data_points_test))


# Check if same data exists in different parts of the split
for data_point in data_points:
    if (data_point in data_points_train) and (data_point in data_points_validation):
        print("ERROR!!!")
    if (data_point in data_points_train) and (data_point in data_points_test):
        print("ERROR!!!")
    if (data_point in data_points_validation) and (data_point in data_points_test):
        print("ERROR!!!")
# Check if there are duplicates in a set
print(any(data_points_train.count(x) > 1 for x in data_points_train))
print(any(data_points_validation.count(x) > 1 for x in data_points_validation))
print(any(data_points_test.count(x) > 1 for x in data_points_test))

0.5172131147540984
631
train_size: 0.7179080824088748, test_size: 0.13946117274167988, validation_size: 0.14263074484944532
589
train_size: 0.7402376910016978, test_size: 0.12563667232597622, validation_size: 0.13412563667232597
train_size: 0.728688524590164, test_size: 0.13278688524590163, validation_size: 0.13852459016393442
889 169 162
False
False
False


In [ ]:
print(len(data_points_train), len(data_points_validation), len(data_points_test))

889 169 162


In [ ]:
random.seed(1839)

# Processing the images

img_size = 512
pickle_chunk_size = 5000

def load_and_process_image(file_path, img_size):
    try:
        # image = Image.open(file_path)
        # image.save('./image_sample.png')

        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        # cv2.imwrite('./image_sample.png', image)

        image = (image / 256).astype(np.uint8)  # 16-bit to 8-bit
        image = Image.fromarray(image)
        # image.save('./image_sample.png')

        shorter_edge = min(image.size)

        # transforms.ToTensor() is removed for the pickle size to be smaller
        preprocess = transforms.Compose([
            transforms.CenterCrop(shorter_edge),
            transforms.Resize((img_size, img_size))
        ])

        image = preprocess(image)
        return image

    except Exception as e:
        print(f'Error loading the image: {e}')
        return None


def chunks(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i+chunk_size]


mode = 'validation'
data_points = data_points_validation

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH25/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)

mode = 'testing'
data_points = data_points_test

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH25/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)

mode = 'training'
data_points = data_points_train

data_points_chunks_generator = chunks(data_points, pickle_chunk_size)

output_dir = os.path.join(f'./processed_data/HH25/{mode}')
Path(output_dir).mkdir(parents=True, exist_ok=True)

for chunk_no, chunk in enumerate(data_points_chunks_generator):
    dataset_part = list()
    for chunk_entry_no, chunk_entry in enumerate(chunk):
        if (chunk_entry_no + 1) % 10 == 0:
            print(f'Processing chunk entry no: {chunk_entry_no + 1}')

        image = load_and_process_image(chunk_entry['path'], img_size)
        data_point = {'label': chunk_entry['label'], 'image': image, 'additional_info': chunk_entry['additional_info']}
        dataset_part.append(data_point)

    with open(os.path.join(output_dir, f'{mode}_{img_size}_part_{chunk_no + 1}.pickle'), 'wb') as f:
        pickle.dump(dataset_part, f)


Processing chunk entry no: 10
Processing chunk entry no: 20
Processing chunk entry no: 30
Processing chunk entry no: 40
Processing chunk entry no: 50
Processing chunk entry no: 60
Processing chunk entry no: 70
Processing chunk entry no: 80
Processing chunk entry no: 90
Processing chunk entry no: 100
Processing chunk entry no: 110
Processing chunk entry no: 120
Processing chunk entry no: 130
Processing chunk entry no: 140
Processing chunk entry no: 150
Processing chunk entry no: 160
Processing chunk entry no: 10
Processing chunk entry no: 20
Processing chunk entry no: 30
Processing chunk entry no: 40
Processing chunk entry no: 50
Processing chunk entry no: 60
Processing chunk entry no: 70
Processing chunk entry no: 80
Processing chunk entry no: 90
Processing chunk entry no: 100
Processing chunk entry no: 110
Processing chunk entry no: 120
Processing chunk entry no: 130
Processing chunk entry no: 140
Processing chunk entry no: 150
Processing chunk entry no: 160
Processing chunk entry no:

In [ ]:
import pickle
import os

def load_pickle(filepath):
  with open(filepath, 'rb') as f:
    data = pickle.load(f)
  return data


file_to_load = './training_512_part_1.pickle'
train_hh19 = load_pickle(file_to_load)

file_to_load = './testing_512_part_1.pickle'
test_hh25 = load_pickle(file_to_load)

file_to_load = './validation_512_part_1.pickle'
validation_hh25 = load_pickle(file_to_load)


print(train_hh19[0])
print(test_hh25[0])
print(validation_hh25[0])

{'label': 0, 'image': <PIL.Image.Image image mode=L size=512x512 at 0x7C4900004340>, 'additional_info': {'id': 16, 'grade': 5, 'real': 1, 'stage': 19}}
{'label': 0, 'image': <PIL.Image.Image image mode=L size=512x512 at 0x7C4900007BE0>, 'additional_info': {'id': 52, 'grade': 5, 'real': 1, 'stage': 25}}
{'label': 0, 'image': <PIL.Image.Image image mode=L size=512x512 at 0x7C4900238700>, 'additional_info': {'id': 66, 'grade': 4, 'real': 1, 'stage': 25}}


In [ ]:
train_hh19_ids = [data_point['additional_info']['id'] for data_point in train_hh19]
test_hh25_ids = [data_point['additional_info']['id'] for data_point in test_hh25]
validation_hh25_ids = [data_point['additional_info']['id'] for data_point in validation_hh25]

common_items = set(train_hh19_ids) & set(test_hh25_ids)

if common_items:
    print("Common items found:", common_items)
else:
    print("No common items found")